In [0]:
from __future__ import division
import numpy as np
import pandas as pd
import os, sys,time
import vcf
import gzip
from Bio import SeqIO
from collections import OrderedDict, Counter
import random


In [0]:
vcftools = "/home/lindb/g/src/vcftools_0.1.13/bin/vcftools"
bcftools = "/home/lindb/g/src/bcftools-1.3/bcftools"
tabix = "/home/lindb/g/src/htslib-1.3/tabix"
bgzip = "/home/lindb/g/src/htslib-1.3/bgzip"
plink = "/home/lindb/g/src/plink-1.07-x86_64/plink --noweb"
java  = "/home/lindb/g/src/jre1.8.0_73/bin/java"

In [0]:
snp_dir = '/home/lindb/wbp/concatenated/snps/'
vcf_file = os.path.join(snp_dir,'samtools_1.3.vcf.gz')

In [0]:
assert os.path.exists(vcf_file)

In [0]:
!$vcftools --remove-indels \
--max-missing 0.5 \
--min-alleles 2 \
--max-alleles 2 \
--remove-filtered-all \
--recode \
--recode-INFO-all \
--gzvcf \
$vcf_file \
--out $vcf_file

In [0]:
vcf_filtered = "%s.recode.vcf" % vcf_file
vcf_filtered_gz = "%s.gz" % vcf_filtered

In [0]:
!$bgzip -c $vcf_filtered > {vcf_filtered_gz}
!$tabix {vcf_filtered_gz}

In [0]:
def get_vcf_stats(vcf_gz):
    
    stats = ['depth',
            'site-depth',
            'site-mean-depth',
            'site-quality',
            'missing-indv',
            'missing-site',
            'freq',
            'counts',
            'hardy',
            'het']
    
    for stat in stats:
        !$vcftools --gzvcf $vcf_gz \
        --out $vcf_gz \
        {"--%s" % stat} 

In [0]:
get_vcf_stats(vcf_filtered_gz)

In [0]:
files = os.listdir(snp_dir)

In [0]:
[os.path.join(snp_dir,f) for f in files if '.l' in f and 'log' not in f]

In [0]:
pd.set_option('display.max_columns', 100)

def get_MAF(row):
    try:
        return np.min([row.A1_freq, row.A2_freq])
    except:
        print(row)
        
def get_correction(n):
    #for finite sample size
    return (2*n)/(2*n-1)

def calculate_Fis(vals):
    try:
        data = [float(x) for x in vals.split("/")]
        assert len(data) == 3
        num_individuals = np.sum(data)
        total_alleles = 2*num_individuals
        a1_count = 2*data[0]
        a2_count = 2*data[2]
        het_count = data[1]
        a1_count += het_count
        a2_count += het_count
        a1_freq = a1_count/total_alleles
        a2_freq = a2_count/total_alleles
        assert a1_freq + a2_freq == 1.0
        He = 2 * a1_freq * a2_freq * get_correction(num_individuals)
        Ho = het_count/num_individuals
        Fis = 1 - (Ho/He)
        return Fis
    except:
        return -9

def combine_vcf_stats(filedir, prefix):
    #hardy_files = !ls {filedir}/{prefix}*.hwe
    print "hardy files"
    files = os.listdir(filedir)
    hardy_files = [os.path.join(filedir,f) for f in files if f.endswith('hwe')]
    hardy = pd.read_csv(hardy_files[0], sep="\t")

    hardy.columns = ['CHROM', 'POS', 'OBS(HOM1/HET/HOM2)', 'E(HOM1/HET/HOM2)', 'ChiSq_HWE',
       'P_HWE', 'P_HET_DEFICIT', 'P_HET_EXCESS']
    hardy.index = hardy.apply(lambda x: "%s-%d" % (x.CHROM, x.POS), axis=1)
    
    print "loci files"
    #loci_files = !ls {filedir}/{prefix}*.l* | grep -v log
    loci_files = [os.path.join(filedir,f) for f in files if '.l' in f and 'log' not in f]
    loci_df = pd.concat([pd.read_csv(x, sep="\t", skiprows=0) for x in loci_files], axis=1)
    chrom_pos = loci_df.ix[:,0:2]
    
    print "frq files"
    #frq_files = !ls {filedir}/{prefix}*.frq* | grep -v count
    frq_files = [os.path.join(filedir,f) for f in files if f.endswith('frq')]
    frq_data = []
    h = open(frq_files[0])
    header = h.readline().strip().split()
    for line in h:
        frq_data.append(line.strip().split('\t'))

    header = ['CHROM', 'POS', 'N_ALLELES', 'N_CHR', 'A1_FREQ', "A2_FREQ"]
    frq_df = pd.DataFrame(frq_data)
    print(frq_df.columns)
    #frq_df = frq_df.drop([6,7],axis=1)
    frq_df.columns = header
    frq_df.index = frq_df.apply(lambda x: "%s-%s" % (x.CHROM, x.POS), axis=1)
    
    print "loci df"
    loci_df = loci_df.drop(['CHROM','CHR','POS'], axis=1)
    loci_df = pd.concat([chrom_pos, loci_df], axis=1)
    loci_df.index = loci_df.apply(lambda x: "%s-%d" % (x.CHROM, x.POS), axis=1)
    
    print "identifying alleles"
    loci_df = pd.concat([loci_df, frq_df, hardy], axis=1)
    loci_df["A1_allele"] = loci_df.apply(lambda row: row.A1_FREQ.split(":")[0], axis=1)
    loci_df["A2_allele"] = loci_df.apply(lambda row: row.A2_FREQ.split(":")[0], axis=1)
    
    print "calculating allele freqs"
    loci_df["A1_freq"] = loci_df.apply(lambda row: float(row.A1_FREQ.split(":")[1]), axis=1)
    loci_df["A2_freq"] = loci_df.apply(lambda row: float(row.A2_FREQ.split(":")[1]), axis=1)
    
    print "getting MAF"
    loci_df['MAF'] = loci_df.apply(get_MAF, axis=1)
    loci_df = loci_df.drop(['CHROM', 'POS'], axis=1)
    
    print "calculating FIS"
    loci_df['Fis'] = loci_df['OBS(HOM1/HET/HOM2)'].apply(calculate_Fis)
    
    print 'done'
    return loci_df, frq_df, hardy

In [0]:
snp_dir

In [0]:
loci_df, frq_df, hardy = combine_vcf_stats(snp_dir, "samtools")

In [0]:
loci_df.head()

In [0]:
frq_df.head()

In [0]:
hardy.head()

In [0]:
loci_file = '/home/lindb/wbp/concatenated/snps/loci_vcf_stats.txt'
loci_df.to_csv(loci_file,header=True,index=True,sep="\t")

In [0]:
frq_file = '/home/lindb/wbp/concatenated/snps/frq_vcf_stats.txt'
frq_df.to_csv(frq_file,header=True,index=True,sep="\t")

In [0]:
hardy_file = '/home/lindb/wbp/concatenated/snps/hwe_vcf_stats.txt'
hardy.to_csv(hardy_file,header=True,index=True,sep="\t")

# Impute with Beagle

    cd /home/lindb/wbp/concatenated/snps/beagle40
    ln -s ../samtools_1.3.vcf.gz.recode.vcf.gz
    /home/lindb/g/src/jdk1.8.0_73/bin/java -jar ~/g/src/BEAGLE4/beagle.r1399.jar \
    gl=samtools_1.3.vcf.gz.recode.vcf.gz \
    out=beagle40 \
    nthreads=32 \
    phase-its=20 \
    burnin-its=20 \
    impute-its=20

In [0]:
beagle_dir = os.path.join(snp_dir, "beagle40")

In [0]:
beagle_vcf_gz = os.path.join(beagle_dir, "beagle40.vcf.gz")
assert os.path.exists(beagle_vcf_gz)

In [0]:
get_vcf_stats(beagle_vcf_gz)

In [0]:
beagle_dir

In [0]:
loci_df_beagle, freq_df_beagle, hardy_beagle = combine_vcf_stats(beagle_dir, "beagle40")

In [0]:
loci_df_beagle

In [0]:
freq_df_beagle.head()

In [0]:
hardy_beagle.head()

In [0]:
loci_beagle_file = os.path.join(beagle_dir,'loci_beagle_vcf_stats.txt')
loci_df_beagle.to_csv(loci_beagle_file,header=True,index=True,sep="\t")

In [0]:
freq_beagle = os.path.join(beagle_dir,'frq_beagle_vcf_stats.txt')
freq_df_beagle.to_csv(freq_beagle,header=True,index=True,sep="\t")

In [0]:
hardy_beagle_file = os.path.join(beagle_dir,'hwe_beagle_vcf_stats.txt')
hardy_beagle.to_csv(hardy_beagle_file,header=True,index=True,sep="\t")

In [0]:
chroms = sorted(set([x.split("-")[0] for x in loci_df.index]))

In [0]:
snp_dir

In [0]:
with open(os.path.join(snp_dir, "chrom_map.txt"), "w") as o:
    for i, c in enumerate(chroms):
        o.write("%s\t%d\n" % (c, i))

In [0]:
def write_plink_files(vcf_gz):
    !$vcftools --gzvcf {vcf_gz} \
    --out {vcf_gz} \
    --plink \
    --chrom-map {os.path.join(snp_dir, "chrom_map.txt")}

In [0]:
vcf_filtered_gz

In [0]:
write_plink_files(vcf_filtered_gz)

In [0]:
def write_plink_recode(vcf_gz):
    !$plink --recodeA --tab --file {vcf_gz} --out {vcf_gz}_recodeA

In [0]:
write_plink_recode(vcf_filtered_gz)

In [0]:
loci_df.SUM_DEPTH.describe()

In [0]:
import matplotlib.pyplot as plt
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
loci_df.QUAL.plot(kind="hist")

In [0]:
len(loci_df[loci_df.Fis == -9])

In [0]:
len(loci_df[loci_df.QUAL >= 10]) - len(loci_df[loci_df.QUAL >= 20])

In [0]:
len(loci_df[loci_df.QUAL < 20]), len(loci_df[loci_df.QUAL < 10])

In [0]:
len(loci_df[loci_df.Fis >= 0.5]), len(loci_df[loci_df.Fis <= -0.5]), len(loci_df[loci_df.MAF < 0.01])

In [0]:
def filter_snps(df, imputed=False):
    if imputed:
        return df[(df.MAF >= 0.01) & 
                  (df.Fis < 0.5) & 
                  (df.Fis > -0.5)]
    else:
        return df[(df.SUM_DEPTH >= 100) & 
                  (df.SUM_DEPTH < 1500) & 
                  (df.QUAL >= 20) & 
                  (df.MAF >= 0.01) & 
                  (df.Fis < 0.5) & 
                  (df.Fis > -0.5)]

In [0]:
loci_stage1 = filter_snps(loci_df)
loci_stage1.shape

In [0]:
loci_stage1.head()

In [0]:
beagle_stage1 = filter_snps(loci_df_beagle, imputed=True)
beagle_stage1.shape

In [0]:
with open(os.path.join(snp_dir, "stage1_positions.txt"), "w") as o:
    for elem in loci_stage1.index:
        o.write("%s\n" % "\t".join(elem.split("-")))
        
with open(os.path.join(beagle_dir, "stage1_positions.txt"), "w") as o:
    for elem in beagle_stage1.index:
        o.write("%s\n" % "\t".join(elem.split("-")))

In [0]:
for d, vcf_gz in zip([snp_dir, beagle_dir], [vcf_filtered_gz, beagle_vcf_gz]):
    !$vcftools --gzvcf $vcf_gz \
    --remove-indels  \
    --remove-filtered-all \
    --recode \
    --recode-INFO-all \
    --positions {os.path.join(d, "stage1_positions.txt")} \
    --out {os.path.join(d, "good_snps")}

In [0]:
for d in [snp_dir, beagle_dir]:
    snps = os.path.join(d, "good_snps.recode.vcf")
    snps_gz = snps + ".gz"
    !$bgzip -c {snps} > {snps_gz}
    !$tabix {snps_gz}

In [0]:
def get_intersection(imp, ni):
    return set.intersection(set(ni.index), set(imp.index))

In [0]:
isect = get_intersection(beagle_stage1, loci_stage1)

In [0]:
isect = sorted(isect)

In [0]:
len(loci_stage1.index), len(beagle_stage1.index), len(isect)

In [0]:
for d in [snp_dir, beagle_dir]:
    with open(os.path.join(d, "isect_positions.txt"), "w") as o:
        for elem in isect:
            o.write("%s\n" % "\t".join(elem.split("-")))

In [0]:
for d, vcf_gz in zip([snp_dir, beagle_dir], [vcf_filtered_gz, beagle_vcf_gz]):
    !$vcftools --gzvcf $vcf_gz \
    --remove-indels  \
    --remove-filtered-all \
    --recode \
    --recode-INFO-all \
    --positions {os.path.join(d, "isect_positions.txt")} \
    --out {os.path.join(d, "isect_snps")}

In [0]:
vcfsort = '/home/lindb/g/src/vcftools_0.1.13/perl/vcf-sort'

In [0]:
for d in [snp_dir, beagle_dir]:
    snps = os.path.join(d, "isect_snps.recode.vcf")
    snps_gz = snps + ".gz"
    !$bgzip -c {snps} > {snps_gz}
    !$tabix {snps_gz}
    
    srted = snps_gz + "_sorted.vcf"
    srted_gz = srted + ".gz"
    !$vcfsort {snps_gz} > {srted}
    !$bgzip -c {srted} > {srted_gz}
    !$tabix {srted_gz}

# choose one snp per contig

In [0]:
snp_dir,beagle_dir

In [0]:
#because I want to go to bed and this will take a while with 713K recs in vcf:
pytext = '''from __future__ import division
import numpy as np
import pandas as pd
import os, sys,time
import vcf
from collections import OrderedDict, Counter
import random

VCFmissing = '/home/lindb/wbp/concatenated/snps/isect_snps.recode.vcf'
missing_reader = vcf.Reader(open(VCFmissing),'r')

filE1 = '/home/lindb/wbp/concatenated/snps/update.txt'
if os.path.exists(filE1):
    os.remove(filE1)
with open(filE1,'w') as o:
    text = 'starting recs \\n'
    o.write("%s" % text)


locDict = OrderedDict()
loccount = 0
for rec in missing_reader:
    contig = rec.CHROM
    locus = rec.POS
    if contig not in locDict.keys():
        locDict[contig] = Counter()

    womp = 0
    for samp in rec.samples:
        gt = samp['GT']

        if '.' in gt:
            try:
                assert gt == './.'
                womp += 1 #womp womp
            except Exception as E:
                print gt,E
        else: #add count for a sample without missing data
            locDict[contig][str(locus)] += 1

    assert womp + locDict[contig][str(locus)] == len(rec.samples)
    loccount += 1
    if loccount % 1000 == 0:
        with open(filE1,'a') as o:
            o.write("%s\\n" % str(loccount))
    #break

with open(filE1,'a') as o:
    o.write("choosing snps \\n")

keep = OrderedDict()
contcount = 0
for contig in locDict.keys():
    #print contig
    #print len(locDict[contig].keys())

    #find SNP with least missing data
    mx = 0
    mxloc = ['Hello How Are You?']
    for locus in locDict[contig].keys():
        #print locus,locDict[contig][locus]

        if locDict[contig][locus] > mx: #if > max count
            mx = locDict[contig][locus]
            mxloc = [str(locus)]
            #print mxloc
        elif locDict[contig][locus] == mx: #if = max count
            mxloc.append(str(locus))
            #print mxloc

    assert mxloc != ['Hello How Are You?']
    if len(mxloc) > 1:
        x = random.random()

        bins = []
        for i in range(len(mxloc)):
            if i == 0:
                bins.append(0)
            else:
                bins.append(i/len(mxloc))
        bins.append(1)

        for b in range(len(bins)):
            if (x > bins[b]) and (x <= bins[b+1]):
                lowcuss = mxloc[b]
                #print "b=",b
                try:
                    print snp
                    sys.exit("snp already exists. WTF?")
                except:
                    pass
                snp = "-".join([contig,str(lowcuss)])
                break #keep
    else:
        snp = "-".join([contig,str(mxloc[0])])
    #print snp

    assert contig not in keep.keys()
    keep[contig] = snp
    
    #delete just in case
    del snp, locus
    try:
        del lowcuss
    except:
        pass
    contcount += 1
    if contcount % 1000 == 0:
        with open(filE1,'a') as o:
            o.write("%s" % str(contcount))

with open(filE1,'a') as o:
    o.write("writing file\\n")

filE = '/home/lindb/wbp/concatenated/snps/one_snp_per_contig.txt'
if os.path.exists(filE):
    os.remove(filE)
with open(filE,'w') as o:
    for k in keep.keys():
        text = '\\t'.join(keep[k].split("-")) + '\\n'
        o.write("%s\\n" % text)

'''
filE = '/home/lindb/wbp/concatenated/snps/get_one_snp_per_contig.py'
with open(filE,'w') as o:
    o.write("%s" % pytext)

In [0]:
shtext = '''#!/bin/bash
#$ -N contigs
#$ -V
#$ -j y
#$ -cwd

cd /home/lindb/wbp/concatenated/snps/
python get_one_snp_per_contig.py

'''
filE = '/home/lindb/wbp/concatenated/snps/run_get_one_snp_per_contig.sh'
with open(filE,'w') as o:
    o.write("%s" % shtext)

In [0]:
cd ~/wbp/concatenated/snps/

In [0]:
!qsub $filE

In [0]:
import shutil

In [0]:
shutil.copy('/home/lindb/wbp/concatenated/snps/one_snp_per_contig.txt',
            '/home/lindb/wbp/concatenated/snps/beagle40/one_snp_per_contig.txt')

In [0]:
df = pd.read_csv('/home/lindb/wbp/concatenated/snps/one_snp_per_contig.txt',header=None,sep="\t")
df.head()

In [0]:
df.shape

In [0]:
len(np.unique(df[0]))

In [0]:
len(df.index)

In [0]:
[vcf_filtered_gz, beagle_vcf_gz]

In [0]:
vcf_filtered_gz_safe = '/home/lindb/wbp/concatenated/snps/onepcontig/samtools_1.3.vcf.gz.recode.vcf.gz'
beagle_vcf_gz_safe = '/home/lindb/wbp/concatenated/snps/onepcontig/beagle40.vcf.gz'
assert os.path.exists(vcf_filtered_gz_safe)
assert os.path.exists(beagle_vcf_gz_safe)

In [0]:
#write files
for d, vcf_gz in zip([snp_dir, beagle_dir], [vcf_filtered_gz_safe, beagle_vcf_gz_safe]):
    !$vcftools --gzvcf $vcf_gz \
    --remove-indels  \
    --remove-filtered-all \
    --recode \
    --recode-INFO-all \
    --positions {os.path.join(d, "one_snp_per_contig.txt")} \
    --out {os.path.join(d, "isect_one_per_contig")}

In [0]:
#zip files
for d in [snp_dir, beagle_dir]:
    snps = os.path.join(d, "isect_one_per_contig.recode.vcf")
    snps_gz = snps + ".gz"
    !$bgzip -c {snps} > {snps_gz}
    !$tabix {snps_gz}

In [0]:
os.path.exists(snps_gz)

In [0]:
vcfsort

In [0]:
for d in [snp_dir, beagle_dir]:
    snps = os.path.join(d, "isect_one_per_contig.recode.vcf")
    snps_gz = snps + ".gz"
    srted = snps_gz + "_sorted.vcf"
    srted_gz = srted + ".gz"

    !$vcfsort {snps_gz} > {srted}
    !$bgzip -c {srted} > {srted_gz}
    !$tabix {srted_gz}    

In [0]:
for d in [snp_dir, beagle_dir]:
    f = os.path.join(d, "isect_one_per_contig.recode.vcf.gz_sorted.vcf.gz")
    assert os.path.exists(f)
    write_plink_files(f)
    write_plink_recode(f)

In [0]:
for d in [snp_dir, beagle_dir]:
    f = os.path.join(d, "isect_one_per_contig.recode.vcf.gz_sorted.vcf.gz")
    assert os.path.exists(f)
    !$vcftools --gzvcf {f} \
    --out {f} \
    --012

In [0]:
VCFmissing = os.path.join(snp_dir,'isect_snps.recode.vcf')
missing_reader = vcf.Reader(open(VCFmissing),'r')

locDict = OrderedDict()
for rec in missing_reader:
    print rec
    contig = rec.CHROM
    locus = rec.POS
    if contig not in locDict.keys():
        locDict[contig] = Counter()

    womp = 0
    for i,samp in enumerate(rec.samples): 
        gt = samp['GT']

        if '.' in gt:
            try:
                assert gt == './.'
                womp += 1 #womp womp
            except Exception as E:
                print gt,E
        else: #add count for a sample without missing data
            locDict[contig][str(locus)] += 1

    assert womp + locDict[contig][str(locus)] == len(rec.samples)
    break

keep = OrderedDict()
for contig in locDict.keys():
    #print contig
    #print len(locDict[contig].keys())

    #find SNP with least missing data
    mx = 0
    mxloc = ['Hello How Are You?']
    for locus in locDict[contig].keys():
        #print locus,locDict[contig][locus]

        if locDict[contig][locus] > mx: #if > max count
            mx = locDict[contig][locus]
            mxloc = [str(locus)]
            #print mxloc
        elif locDict[contig][locus] == mx: #if = max count
            mxloc.append(str(locus))
            #print mxloc

    assert mxloc != ['Hello How Are You?']
    if len(mxloc) > 1:
        x = random.random()

        bins = []
        for i in range(len(mxloc)):
            if i == 0:
                bins.append(0)
            else:
                bins.append(i/len(mxloc))
        bins.append(1)

        for b in range(len(bins)):
            if (x > bins[b]) and (x <= bins[b+1]):
                lowcuss = mxloc[b]
                #print "b=",b
                try:
                    print snp
                    sys.exit("snp already exists. WTF?")
                except:
                    pass
                snp = "-".join([contig,str(lowcuss)])
                break #keep
    else:
        snp = "-".join([contig,str(mxloc[0])])
    #print snp

    assert contig not in keep.keys()
    keep[contig] = snp
    
    #delete just in case
    del snp, locus
    try:
        del lowcuss
    except:
        pass

filE = '/home/lindb/wbp/concatenated/snps/one_snp_per_contig.txt'
if os.path.exists(filE):
    os.remove(filE)
with open(filE,'w') as o:
    for k in keep.keys():
        text = '\t'.join(keep[k].split("-")) + '\n'
        o.write("%s" % text)
        
df = pd.read_csv(filE,sep="\t",header=None)
df    